# Markov Decision Process
## Policy Iteration method
### Step 1 :- Applying the grid world

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
class Environment:
    def __init__(self, n, m):
        self.n = n  # Number of rows
        self.m = m  # Number of columns
        self.state = (0, 0)  # Agent's initial position (x, y)
        self.actions = ['Up', 'Down', 'Left', 'Right']
        self.grid = self.grid_world()  # Initialize the grid

    def step(self, action):
        x, y = self.state

        # Calculate the new position based on the action
        if action == 'Up':
            x_new, y_new = x - 1, y
        elif action == 'Down':
            x_new, y_new = x + 1, y
        elif action == 'Left':
            x_new, y_new = x, y - 1
        elif action == 'Right':
            x_new, y_new = x, y + 1
        else:
            raise ValueError("Invalid action")

        # Check if the new position is valid (within bounds and not an obstacle)
        if 0 <= x_new < self.n and 0 <= y_new < self.m and self.grid[x_new, y_new] != -1:
            self.state = (x_new, y_new)

        # Calculate reward
        if self.grid[x_new, y_new] == 2:  # Goal state
            reward = 10
            done = True
        elif self.grid[x_new, y_new] == -1:  # Obstacle
            reward = -10
            done = True
        else:
            reward = 0
            done = False

        return self.state, reward, done

    def reset(self):
        self.state = (0, 0)  # Reset to the start state
        return self.state

    def render(self):
        grid = self.grid.copy()
        x, y = self.state
        grid[x, y] = 9
        print(grid)

    def grid_world(self):
        grid = np.zeros((self.n, self.m), dtype=int)
        start_state = (0, 0)
        goal_state = (self.n - 1, self.m - 1)
        obstacles = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 0), (3, 1), (3, 2)]

        grid[start_state] = 1  # Start state
        grid[goal_state] = 2  # Goal state
        for obstacle in obstacles:
            grid[obstacle] = -1  # Obstacles

        return grid

    def random_policy(self):
        return random.choice(self.actions)

    def value(self):
        value_table = np.zeros((self.n, self.m))


    def value_iteration(self, gamma=0.9, theta=1e-6):
        """
        Value Iteration Algorithm to compute optimal state values.
        """
        V = np.zeros((self.n, self.m))  # Initialize state values
        while True:
            delta = 0
            for x in range(self.n):
                for y in range(self.m):
                    if self.grid[x, y] in [-1, 2]:  # Skip obstacles & goal
                        continue

                    v = V[x, y]
                    values = []
                    for action in self.actions:
                        x_new, y_new = self.next_position(x, y, action)
                        reward = self.get_reward(x_new, y_new)
                        values.append(reward + gamma * V[x_new, y_new])

                    V[x, y] = max(values)
                    delta = max(delta, abs(v - V[x, y]))

            if delta < theta:
                break
        return V

    def policy_iteration(self, gamma=0.9, theta=1e-6):
        """
        Policy Iteration Algorithm: Finds an optimal policy.
        """
        policy = np.random.choice(self.actions, size=(self.n, self.m))
        V = np.zeros((self.n, self.m))

        while True:
            # Policy Evaluation
            while True:
                delta = 0
                for x in range(self.n):
                    for y in range(self.m):
                        if self.grid[x, y] in [-1, 2]:  # Skip obstacles & goal
                            continue

                        v = V[x, y]
                        action = policy[x, y]
                        x_new, y_new = self.next_position(x, y, action)
                        reward = self.get_reward(x_new, y_new)
                        V[x, y] = reward + gamma * V[x_new, y_new]
                        delta = max(delta, abs(v - V[x, y]))

                if delta < theta:
                    break

            # Policy Improvement
            policy_stable = True
            for x in range(self.n):
                for y in range(self.m):
                    if self.grid[x, y] in [-1, 2]:  # Skip obstacles & goal
                        continue

                    old_action = policy[x, y]
                    values = {}
                    for action in self.actions:
                        x_new, y_new = self.next_position(x, y, action)
                        reward = self.get_reward(x_new, y_new)
                        values[action] = reward + gamma * V[x_new, y_new]

                    policy[x, y] = max(values, key=values.get)
                    if old_action != policy[x, y]:
                        policy_stable = False

            if policy_stable:
                break

        return policy, V

    def next_position(self, x, y, action):
        """
        Returns the next position based on action.
        """
        if action == 'Up' and x > 0:
            x -= 1
        elif action == 'Down' and x < self.n - 1:
            x += 1
        elif action == 'Left' and y > 0:
            y -= 1
        elif action == 'Right' and y < self.m - 1:
            y += 1
        return x, y

    def get_reward(self, x, y):
        """
        Returns reward based on grid position.
        """
        if self.grid[x, y] == 2:  # Goal
            return 10
        elif self.grid[x, y] == -1:  # Obstacle
            return -10
        return 0  # Default reward


In [ ]:
env = Environment(5,5)

In [ ]:
env.grid_world()


array([[ 1, -1, -1,  0,  0],
       [ 0,  0, -1, -1,  0],
       [-1,  0,  0,  0,  0],
       [ 0, -1, -1,  0,  0],
       [ 0,  0,  0,  0,  2]])

In [ ]:
policy, values = env.policy_iteration()
print("Optimal Policy:")
print(policy)
print("State Values:")
print(values)

Optimal Policy:
[['Down' 'Right' 'Down' 'Right' 'Down']
 ['Right' 'Down' 'Down' 'Left' 'Down']
 ['Down' 'Right' 'Right' 'Down' 'Down']
 ['Down' 'Up' 'Down' 'Down' 'Down']
 ['Right' 'Right' 'Right' 'Right' 'Left']]
State Values:
[[ 4.782969  0.        0.        6.561     7.29    ]
 [ 5.31441   5.9049    0.        0.        8.1     ]
 [ 0.        6.561     7.29      8.1       9.      ]
 [ 6.561     0.        0.        9.       10.      ]
 [ 7.29      8.1       9.       10.        0.      ]]


In [ ]:
env.render()

[[ 9 -1 -1  0  0]
 [ 0  0 -1 -1  0]
 [-1  0  0  0  0]
 [ 0 -1 -1  0  0]
 [ 0  0  0  0  2]]


In [ ]:
import numpy as np
import random
from queue import Queue

class Environment:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.state = (0, 0)
        self.actions = ['Up', 'Down', 'Left', 'Right']
        self.grid = self.generate_valid_grid()  # Randomized grid with a valid path
        self.path = []  # To store the optimal path

    def generate_valid_grid(self):
        """
        Generates a random grid where obstacles are placed such that there is always a valid path.
        """
        while True:
            grid = np.zeros((self.n, self.m), dtype=int)
            grid[self.n - 1, self.m - 1] = 2  # Goal state

            # Randomly place obstacles (ensuring start and goal are not blocked)
            for _ in range(int(0.2 * self.n * self.m)):  # 20% of cells as obstacles
                x, y = random.randint(0, self.n - 1), random.randint(0, self.m - 1)
                if (x, y) != (0, 0) and (x, y) != (self.n - 1, self.m - 1):
                    grid[x, y] = -1  # Obstacle

            # Check if there's a valid path from (0,0) to (n-1,m-1)
            if self.is_valid_path(grid):
                return grid  # Return only if a path exists

    def is_valid_path(self, grid):
        """
        Uses BFS to check if there is a path from start (0,0) to goal (n-1, m-1).
        """
        q = Queue()
        q.put((0, 0))
        visited = set()
        visited.add((0, 0))

        while not q.empty():
            x, y = q.get()
            if (x, y) == (self.n - 1, self.m - 1):
                return True  # Path found

            for action in self.actions:
                nx, ny = self.next_position(x, y, action)
                if (0 <= nx < self.n and 0 <= ny < self.m and
                        grid[nx, ny] != -1 and (nx, ny) not in visited):
                    q.put((nx, ny))
                    visited.add((nx, ny))

        return False  # No path found

    def step(self, action):
        x, y = self.state
        x_new, y_new = self.next_position(x, y, action)

        # Check if the new position is within bounds
        if 0 <= x_new < self.n and 0 <= y_new < self.m:
            if self.grid[x_new, y_new] != -1:  # Not an obstacle
                self.state = (x_new, y_new)  # Move the agent
                self.path.append((x_new, y_new))

                if self.grid[x_new, y_new] == 2:  # Goal reached
                    return self.state, 10, True  # Reward, Done = True

                return self.state, 0, False  # Reward, Done = False

        return self.state, -1, False  # Hitting a wall gives a small penalty

    def next_position(self, x, y, action):
        """
        Determines the next position given an action.
        """
        if action == 'Up':
            return x - 1, y
        elif action == 'Down':
            return x + 1, y
        elif action == 'Left':
            return x, y - 1
        elif action == 'Right':
            return x, y + 1
        return x, y  # No movement if invalid action

    def render(self):
        """
        Displays the grid using textual symbols.
        """
        display_grid = np.full((self.n, self.m), '.', dtype=str)

        # Mark obstacles
        for x in range(self.n):
            for y in range(self.m):
                if self.grid[x, y] == -1:
                    display_grid[x, y] = '🚧'
                elif self.grid[x, y] == 2:
                    display_grid[x, y] = '🏁'

        # Mark the path taken
        for x, y in self.path:
            display_grid[x, y] = '🟩'

        # Mark the agent
        x, y = self.state
        display_grid[x, y] = '🤖'

        # Print the grid
        for row in display_grid:
            print(" ".join(row))
        print("\n")

    def find_path(self):
        """
        Finds an optimal path using BFS.
        """
        q = Queue()
        q.put((0, 0, []))  # Store the path in the queue
        visited = set()
        visited.add((0, 0))

        while not q.empty():
            x, y, path = q.get()
            path.append((x, y))

            if (x, y) == (self.n - 1, self.m - 1):
                self.path = path  # Store the final path
                return path  # Return the optimal path

            for action in self.actions:
                nx, ny = self.next_position(x, y, action)
                if (0 <= nx < self.n and 0 <= ny < self.m and
                        self.grid[nx, ny] != -1 and (nx, ny) not in visited):
                    q.put((nx, ny, path[:]))  # Store path progression
                    visited.add((nx, ny))

        return []  # No path found

    def run(self):
        """
        Runs an agent to reach the goal using BFS path-finding.
        """
        path = self.find_path()
        for pos in path:
            self.state = pos
            self.render()  # Show the movement of the agent
            if pos == (self.n - 1, self.m - 1):
                print("🎉 Goal Reached! 🎉\n")
                break


# Example Usage
env = Environment(6, 6)  # Create a 6x6 environment
env.render()  # Display the initial state
env.run()  # Run the agent to the goal
